In [1]:
import os
import time

import keras
import keras.layers as kl
import keras.models as km
import keras.utils as ku
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.color as skc
import tensorflow as tf
from keras.applications.vgg19 import VGG19
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from skimage.io import imsave, imshow
from skimage.transform import resize
from tqdm import tqdm

In [2]:
from train_autoencoder_vgg19 import predict_grayscale_to_rgb

In [3]:
# load encoder model
encoder_model = km.load_model('../models/encoder/vgg19_encoder.h5')

In [ ]:
encoder_model.summary()

In [10]:
decoder_model = km.load_model('../models/decoder/decoder_model_vgg19_5/decoder_model_vgg19_5.h5')

In [11]:
decoder_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 256)       1179904   
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       295040    
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        36928     
                                                                 
 up_sampling2d (UpSampling2D  (None, 28, 28, 64)       0         
 )                                                               
                                                                 
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        18464     
                                                        

In [ ]:
TEST_PATH = '../data/imagenet_data/test/color'
step = 100
files = [os.path.join(TEST_PATH, f) for f in os.listdir(TEST_PATH)][step:step+100]
preds = predict_grayscale_to_rgb(files, encoder_model, decoder_model)

# display images in grid
fig, ax = plt.subplots(10, 10, figsize=(10, 10))
for i, rgb_img in enumerate(preds):
    ax[i//10, i%10].imshow(rgb_img)
    ax[i//10, i%10].axis('off')